# Inference Named Entity Recognition with SpaCy and 🔭 Galileo

In this tutorial, we'll log an inference dataset on a pre-trained model with SpaCy and explore the results in Galileo.

In [1]:
#@title Install `dataquality` and other dependencies

# Upgrade pip
!pip install -U pip &> /dev/null
# Install HF datasets for downloading the example datasets
!pip install -U dataquality datasets "spacy==3.2.1" &> /dev/null

# Login to Galileo

In [2]:
import os

os.environ["GALILEO_CONSOLE_URL_ENTERPRISE"] = "https://console.enterprise-a.rungalileo.io"
os.environ["GALILEO_USERNAME_ENTERPRISE"] = "david@rungalileo.io"
os.environ["GALILEO_PASSWORD_ENTERPRISE"] = "ufz2ytz@eby2MRF2djy"

In [3]:
import os

import dataquality as dq
import spacy

# TODO: Set enterprise env variables
os.environ["GALILEO_CONSOLE_URL"] = os.getenv("GALILEO_CONSOLE_URL_ENTERPRISE")  # Update
os.environ["GALILEO_USERNAME"] = os.getenv("GALILEO_USERNAME_ENTERPRISE")  # Update
os.environ["GALILEO_PASSWORD"] = os.getenv("GALILEO_PASSWORD_ENTERPRISE")  # Update

dq.configure()

📡 https://console.enterprise-a.rungalileo.io
🔭 Logging you into Galileo

🚀 You're logged in to Galileo as david@rungalileo.io!


# Inference DataSet Preparation 
We load text samples from HuggingFace🤗 dataset. This data can be thought of as production data that our model is making predictions on.

In [4]:
#@title 🤗 HuggingFace Dataset
#@markdown You can select any dataset from [here](https://huggingface.co/datasets?language=language:en&task_categories=task_categories:token-classification&task_ids=task_ids:named-entity-recognition&sort=downloads).

dataset_name = 'conllpp' #@param ["wnut_17", "conllpp", "wikiann"] {allow-input: true}
print(f"You selected the {dataset_name} dataset")

from IPython.utils import io
from tqdm import tqdm
from datasets import load_dataset, get_dataset_config_names

# Try to load the data. If a config (subset) is needed, pick one
try:
  with io.capture_output() as captured:
    data = load_dataset(dataset_name)
except ValueError as e:
  if "Config name is missing" not in repr(e):
    raise e

  configs = get_dataset_config_names(dataset_name)
  print(f"The dataset {dataset_name} has multiple subsets {configs}.")
  config = input(f"🖖 Enter the name of the subset to pick (or leave blank for any): ")
  if config:
    assert config in configs, f"{config} is not a valid subset"
  else:
    config = configs[0]
  with io.capture_output() as captured:
    data = load_dataset(dataset_name, name=config)

# Check that the dataset has at least a validation/test set to use as inference data
assert {"validation", "test"}.intersection(data), \
f"💾 The dataset {dataset_name} has no validation or test splits, select another one."

print(f"\n🏆 Dataset {dataset_name} loaded succesfully")

# A small function for minimizing the dataset for testing
import os

def _minimize_for_ci() -> bool:
    return True
    # return os.getenv("MINIMIZE_FOR_CI", "false") == "true"

You selected the conllpp dataset

🏆 Dataset conllpp loaded succesfully


In [5]:
def get_inference_docs(data, nlp) -> list:
    # Get inference split name
    inf_split_name = "validation" if "validation" in data else "test"
    tokens = data[inf_split_name]['tokens']
    text_inputs = [' '.join(_tokens) for _tokens in tokens]

    if _minimize_for_ci():
        text_inputs = text_inputs[:50]

    # Get inference docs
    inference_docs = [nlp.make_doc(text) for text in text_inputs]

    return inference_docs

# Load the model

Inference data can be logged as part of an existing run with previously logged training, validation, and test splits, or it can be logged in isolation.

If you want to access the full suite of inference features, including automated drift detection, first run the [NER Spacy and Galileo training notebook](https://colab.research.google.com/github/rungalileo/examples/blob/main/examples/named_entity_recognition/Named_Entity_Recognition_with_SpaCy_and_%F0%9F%94%AD_Galileo.ipynb#scrollTo=QkUoYnK0oFK_).

Otherwise, we will download a small pretrained model to use for inference.

In [6]:
import spacy

inference_only = True #@param [True, False] {allow-input: false}

if inference_only:
    # Download a small English model for running inference
    !python -m spacy download en_core_web_sm &> /dev/null
    nlp = spacy.load("en_core_web_sm")
else:
    # Load trained model from NER Spacy training notebook
    nlp = spacy.load("my_model")  # TODO: Update path to model

# Inference with Galileo
Inference samples are logged to Galileo using `log_input_docs`. Model data is logged by wrapping the `nlp` object using `watch`. This automatically logs the logits and embeddings from your model to Galileo with just 1 line of code. 

We complete the inference pipeline by setting the split to `inference` and then passing inference Doc objects through our `nlp` model. To complete logging, we call `dq.finish()` after logging inference data.

Each inference run must have a unique `inference_name` that must be consistent for input and model logs. You can log multiple inference datasets with different `inference_name`s on the same Project/Run pair.

**Note:** If you want to connect this inference run to an existing training run, make sure to use the same `project_name` and `run_name`.

In [7]:
import random
from dataquality.integrations.spacy import log_input_docs, unwatch, watch

# 🔭🌕 Initializing a new run in Galileo. Each run is part of a project.
dq.init(task_type="text_ner", 
        project_name="named_entity_recognition_spacy", 
        run_name=f"example_run_{dataset_name.replace('/', '-')}")

inference_docs = get_inference_docs(data, nlp)
inference_name = "example1"
meta = {
    "color": random.choices(["red", "blue", "green"], k=len(inference_docs)),
    "ranking": random.choices(range(1,101), k=len(inference_docs))
}

watch(nlp) # 🔭🌕 One line of Galileo code to capture the model's predictions on the inputs
log_input_docs(inference_docs, inference_name=inference_name, meta=meta) # 🔭🌕 Logging the inference docs with Galileo

dq.set_split("inference", inference_name=inference_name)
for doc in inference_docs:
    nlp(doc)

dq.finish()
unwatch(nlp)

📡 Retrieving run from existing project, named_entity_recognition_spacy
🛰 Connected to project, named_entity_recognition_spacy, and run, example_run_conllpp.
Logging 50 samples [########################################] 100.00% elapsed time  :     0.01s =  0.0m =  0.0h
 

/Users/elliottchartock/Code/examples/.venv/lib/python3.9/site-packages/dataquality/core/init.py:178: UserWarning: Run: named_entity_recognition_spacy/example_run_conllpp already exists! The existing run will get overwritten on call to finish()!
  warnings.warn(
/Users/elliottchartock/Code/examples/.venv/lib/python3.9/site-packages/dataquality/loggers/model_logger/base_model_logger.py:67: GalileoWarning: An error occurred logging this batch, it will be skipped. Error: No samples in this batch had any gold or prediction spans. Logging will be skipped
  warnings.warn(


☁️ Uploading Data


inference:   0%|          | 0/1 [00:00<?, ?it/s]

Processing data for upload:   0%|          | 0/47 [00:00<?, ?it/s]

inference (inf_name=example1):   0%|          | 0/3 [00:00<?, ?it/s]

Uploading data to Galileo:   0%|          | 0.00/57.3k [00:00<?, ?B/s]

Uploading data to Galileo:   0%|          | 0.00/137k [00:00<?, ?B/s]

Uploading data to Galileo:   0%|          | 0.00/22.4k [00:00<?, ?B/s]

Job inference successfully submitted. Results will be available soon at https://console.enterprise-a.rungalileo.io/insights?projectId=13ab66e6-dd33-4d00-b2d1-fb971f97ba2e&runId=5cc6e2dc-e029-4bfc-acee-501b7d53d24d&split=training&metric=f1&depHigh=1&depLow=0&taskType=2
Waiting for job...
	Looking for training data
Done! Job finished with status completed
Click here to see your run! https://console.enterprise-a.rungalileo.io/insights?projectId=13ab66e6-dd33-4d00-b2d1-fb971f97ba2e&runId=5cc6e2dc-e029-4bfc-acee-501b7d53d24d&split=training&metric=f1&depHigh=1&depLow=0&taskType=2
🧹 Cleaning up


In [ ]:
dq.get_data_logger().doc()
help(dq.log_dataset)